# Libraries

In [217]:
import os
import psycopg2
import psycopg2.extras
import datetime
import re
import pandas as pd
import tqdm
import openpyxl
import xlsxwriter
import schedule
import pandas.io.sql as psql
from datetime import timedelta, date
from os.path import basename
import ssl
import smtplib
from email.mime.application import MIMEApplication
from os.path import basename
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.utils import COMMASPACE, formatdate
from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.schedulers.asyncio import AsyncIOScheduler
from apscheduler.schedulers.blocking import BlockingScheduler
from apscheduler.events import EVENT_JOB_EXECUTED, EVENT_JOB_ERROR
# from polyglot.detect import Detector
import pycld2 as cld2
import emoji as emj
import numpy as np

from nltk import word_tokenize
from nltk.util import ngrams

In [218]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

# Connection

In [219]:
def gp_connect():
    try:
#         db = psycopg2.connect(dbname='reports',
#                               user="u",
#                               password="",
#                               host="",
#                               port="")
        return db
    
    except psycopg2.DatabaseError as e:
        print("could not connect to server",e)
today = date.today()
today
conn = gp_connect()

In [220]:
def api_connect_khd():
    try:
#         db = psycopg2.connect(dbname="JDWH",
#                               user="",
#                               password="",
#                               host="",
#                               port="",
#                              sslmode="")
                 # Connect () may also be used a large string parameter,
                 # For example, "host = localhost port = 5432 user = postgres password = postgres dbname = test"
        return db
    except psycopg2.DatabaseError as e:
        print("could not connect to Greenplum server",e)

In [221]:
def api_connect():
    try:
#         db = psycopg2.connect(dbname="",
#                               user="",
#                               password="",
#                               host="",
#                               port="")
                 # Connect () may also be used a large string parameter,
                 # For example, "host = localhost port = 5432 user = postgres password = postgres dbname = test"
        return db
    except psycopg2.DatabaseError as e:
        print("could not connect to Greenplum server",e)
connection_api = api_connect()

# Test

In [468]:
complaints_nim = {'мало', 'приходиться', 'невозможн', 'не возможн', 'не хотеть', 'отказ', 'ужасн', 'зависать', 
                  'нельзя', 'издеваться', 'не корректно', 'из за', 'не получаться', 'не брать', 'не соединять', 
                  'не показывать', 'не одобрить', 'не верный', 'не владеть', 'технический', 'сырой', 'не поступить', 
                  'отвратительный', 'глючить', 'не соединять', 'не начисляться', 'так и не', 'не предусмотреть',
                  'не выйти', 'игнорировать', 'тупой', 'отсутствие', 'не приходить', 'нет ответ', 'не выяснить',
                  'не зачислить', 'не отправлять', 'будто', 'не понятн', 'сожаление', 'не всегда', 'не открываться',
                  'непонятн', 'потеря', 'не уметь', 'мошенник', 'не объяснить', 'бросить', 'тяжело', 'через раз',
                  'не прийти', 'усовер', 'дозваниваться', 'нельзя', 'не компетентный', 'плохой что', 'иногда', 
                  'медленно', 'некорректный', 'не узнать', 'недостоверный', 'грабеж', 'не отображать', 'возмутить',
                  'неудобн', 'нет возможность', 'зачем', 'не обслужить', 'не отобразить', 'не сразу', 'неприятно',
                  'не довольный', 'qr', 'не понять', 'проблемка', 'бесполезный', 'sms', 'раздражать', 'не формироваться', 
                  'проблематичн', 'некомпетентный', 'нет доступ', 'неудобн', 'грабитель', 'неправильн', 'не правильн',
                  'пропадать', 'не гуманный', 'бред', 'постоянно', 'не поступать', 'не понимать', 'отсталый',
                  'невнимательн'
                 }

suggestions_nim = {'доработать', 'предлагать', 'поработать', 'еще бы', 'если быть', 'нужно', 'упростить', 'предлагать',
                   'можно сделать', 'но нужный', 'расширить', 'кроме', 'не помешать', 'жалко что', 'жаль что', 'уменьш', 
                   'сделать более', 'начислять', 'снижение', 'сделать перевод', 'снизить', 'увеличить', 'хороший бы',
                   'ассортимент', 'настроить', 'быть больше'
                  }

In [459]:
conn = api_connect_khd()
query = f"""select * from dwh_rb.dir_nps_reports dnr"""

df = psql.read_sql(query, conn)
conn.close()

#чистим данные
df1 = df.dropna(subset = ['text'])
df1 = df1.astype({'value': 'int64'})
#type transaction может иметь только две оценки 1 и 2 остальное дропаем
df1.drop(df1.loc[(df1['value'] > 2) & (df1['type'] == 'transaction')].index, inplace = True)
df1['text'] = df1['text'].str.replace("\n", " ")
df1['text'] = df1['text'].str.replace(r"[^\w\s]+", " ")
df1['text'] = df1['text'].str.replace(r"\s\s+", " ")
df1['text'] = df1['text'].replace(r'^\s+$|^\t+$|^$', np.nan, regex = True)
df1['text'] = df1['text'].str.lower()
#df1 = df1.replace(r'^\s+$|^\t+$|^$', np.nan, regex = True)
df1 = df1[df1['text'].str.len() > 3]
df1 = df1.drop_duplicates(subset = ['nps_id'], keep = 'first')
df1 = df1.dropna(subset = ['text'])
df1 = df1.reset_index()
del df1['index']


complaints = {'не мочь', 'не смочь', 'не работать', 'не дозвониться', 'не давать', 'другой банк', 'ждать', 'нет возможн',
              'не решить', 'не ответит', 'долго', 'долгий', 'дозвониться до', 'не знать', 'никто не', 'не предупр', 
              'дозвониться', 'расстраивать', 'не одобрять', 'не вид', 'плохо работа', 'удержива', 'сделать пополнение',
              'не перевестись', 'нет нормальный', 'бірақ', 'бирак ', 'не удобн', 'бермейді', 'вообще', 'не отвечать', 
              'не получит', 'не перезвон', 'не доработат', 'маленьк', 'не выдавать', 'проигнорировать', 'не добить', 
              'өкініш', 'не помочь', 'почему', 'не правильно', 'дорого', 'не отображать', 'не брать', 'надоесть', 
              'алмадым', 'бермеди', 'не решать', 'зависать', 'издеваться'}
suggestions = {'хотеться бы', 'быть бы', 'можно быть', 'хотеться', 'исправьте', 'пожалуйст', 'сделать возможность', 
               'улучш', 'просьба', 'надо добавит', 'добав', 'убрать', 'снизить', 'нужно', 'прошу', 'упрости', 'надо',
               'уменьшить', 'исправите'}
product_dict = {1:'MOBILE_APP', 2:'TRANSFERS', 3:'DEBIT_CARD', 4:'LOAN', 5:'PRODUCT_LOAN', 6:'DEPOSIT', 7:'PAYMENTS',
                8:'INVEST', 9:'GARANT', 10:'JUSAN_SHOP', 11:'JMOBILE', 12:'BONUSES', 13:'QR', 14:'CALL_CENTER',
                15:'BRANCH', 16:'EMPLOYEE', 17:'JUNIOR'}
mobile_app = {'прилож'}
transfers = {'перевод'}
debit_card = {'дебет'}
loan = {''}
product_loan = {''}
deposit = {'депозит'}
payments = {'оплат'}
invest = {'инвест'}
garant = {''}
jusan_shop = {'магаз'}
jmobile = {''}
bonuses = {''}
qr = {'киюар', 'qr', 'кюар'}
call_center = {'оператор'}
branch = {''}
employee = {''}
junior = {''}

# dict_kz = {'ә', 'і', 'ң', 'ғ', 'ү', 'ұ', 'қ', 'ө', 'һ', 'тамаша', 'керемет', 'арине'}
# df_ru = pd.DataFrame(columns=['id', 'created_at', 'iin', 'message_id', 'nps_id', 'type', 'value', 'text'])
# df_kz = pd.DataFrame(columns=['id', 'created_at', 'iin', 'message_id', 'nps_id', 'type', 'value', 'text'])

df1['primary_product'] = ''
df1['secondary_product'] = ''
df1['tertiary_product'] = ''
df1['is_bad_review'] = 0
df1['lemmatized'] = 0
#df1['is_bad_review'] = df1['value'].apply(lambda x: True if x < 3 else False)
for i in range(len(df1)):
    if df1['type'][i] == 'transaction':
        if df1['value'][i] == 1:
            df1['is_bad_review'][i] = True
        else:
            df1['is_bad_review'][i] = False
    else:
        if df1['value'][i] < 3:
            df1['is_bad_review'][i] = True
        else:
            df1['is_bad_review'][i] = False

for i in df1.index:
        doc2 = Doc(df1['text'][i])
        doc2.segment(segmenter)
        doc2.tag_morph(morph_tagger)
        lemmatized_text = []
        for token in doc2.tokens:
            token.lemmatize(morph_vocab)
            lemmatized_text.append(token.lemma)
        df1['lemmatized'][i] = lemmatized_text
def listToString(s):
    str1 = ""
    for ele in s:
        str1 += ele + ' '
    return str1
df1['lemmatized_string'] = df1.apply(lambda x: listToString(x['lemmatized']), axis = 1)
df1 = df1[df1['lemmatized_string'].isnull() == False]
df1['bigram_list'] = df1.apply(lambda x: return_bigrams(x['lemmatized_string']), axis = 1)
df1['trigram_list'] = df1.apply(lambda x: return_trigrams(x['lemmatized_string']), axis = 1)
df1['fogram_list'] = df1.apply(lambda x: return_fograms(x['lemmatized_string']), axis = 1)

df1['is_complaint'] = df1['lemmatized_string'].apply(lambda x: True if occurences(x, complaints) else False)
df1['is_suggestion'] = df1['lemmatized_string'].apply(lambda x: True if occurences(x, suggestions) else False)

#проверяем слова в строке на наличие казахских букв/слов
# for i in range(len(df1)):
#     in_dict = any([letter in df1['text'][i] for letter in dict_kz])
#     lang_detector = cld2.detect(df1['text'][i])
#     if in_dict == True or lang_detector[2][0][0] != 'RUSSIAN' or lang_detector[0] == False:
#         df_kz = df_kz.append(df1.iloc[i])
#     else:
#         df_ru = df_ru.append(df1.iloc[i])
        
# df_ru = df_ru.reset_index()
# del df_ru['index']
# df_kz = df_kz.reset_index()
# del df_kz['index']

# df_kaz = df1.loc[df1['text'].str.contains('ә|і|ң|ғ|ү|ұ|қ|ө|һ|тамаша|керемет', regex = True)]
# df1 = df1.loc[df1['text'].str.contains('ә|і|ң|ғ|ү|ұ|қ|ө|һ|тамаша|керемет', regex = True) == False]

/miniconda3/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/tmp/ipykernel_4026/685914342.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df1['text'] = df1['text'].str.replace(r"[^\w\s]+", " ")
/tmp/ipykernel_4026/685914342.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df1['text'] = df1['text'].str.replace(r"\s\s+", " ")
/tmp/ipykernel_4026/685914342.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['is_bad_review'][i] = False
/tmp/ipykernel_4026/685914342.py:86: SettingWithC

In [441]:
df1[['text', 'is_complaint', 'is_suggestion']].tail(50)

,text,is_complaint,is_suggestion
3960,нет перевода с других банков,True,False
3961,научитесь доверять людям,False,False
3962,очень отсталое приложение усовершенствуйте его...,False,False
3963,100,False,False
3964,приложение тормозит очень очень,False,False
3965,очень хорошо,False,False
3966,я жусан парекомендовал сваим друзям они тоже п...,False,False
3967,дали бы денег оценил бы по высшему разряду,False,False
3968,отлично,False,False
3969,ештенке билмейди жасай алмайды,False,False


In [162]:
def occurences(text, words):
    in_dict = any([word in text for word in words])
    if in_dict == True:
        return True
    else:
        return False

In [357]:
def report_maker():
    df_last_50 = df1[['text', 'is_complaint', 'is_suggestion']].tail(50)
    df_last_50.to_excel('Comments_Analysis_last_50.xlsx')

In [61]:
def return_bigrams(text):
    double_word = []
    for grams in ngrams(text.split(), 2):
        double_word.append(grams)
    return double_word
   # double_word = double_word[:-1]
    
def return_trigrams(text):
    triple_word = []
    for grams in ngrams(text.split(), 3):
        triple_word.append(grams)
    return triple_word
   # double_word = double_word[:-1]

def return_fograms(text):
    quadro_word = []
    for grams in ngrams(text.split(), 4):
        quadro_word.append(grams)
    return quadro_word

In [228]:
def counter(df1):
    all_bigrams = []
    for i in df1.index:
        for j in df1['bigram_list'][i]:
            all_bigrams.append(j)

    all_trigrams = []
    for i in df1.index:
        for j in df1['trigram_list'][i]:
            all_trigrams.append(j)

    all_fograms = []
    for i in df1.index:
        for j in df1['fogram_list'][i]:
            all_fograms.append(j)
#     temp_df_bi = pd.DataFrame(all_bigrams)
    temp_df_bi = pd.DataFrame(all_bigrams).value_counts().head(60).to_frame('count').reset_index()
    temp_df_tri = pd.DataFrame(all_trigrams).value_counts().head(30).to_frame('count').reset_index()
    temp_df_four = pd.DataFrame(all_fograms).value_counts().head(30).to_frame('count').reset_index()
#     temp_df_tri = temp_df_tri.value_counts().head(30).to_frame()
#     temp_df_four = temp_df_four.value_counts().head(30).to_frame()
    return temp_df_bi, temp_df_tri, temp_df_four

In [467]:
#берилмейд, мулдем, аз, сөйлесе алмай, төленбейді, берілмеген уменьшить, более

df1[df1['lemmatized_string'].str.contains('частый', regex = False) == True]

,id,created_at,iin,message_id,nps_id,type,value,text,primary_product,secondary_product,tertiary_product,is_bad_review,lemmatized,lemmatized_string,bigram_list,trigram_list,fogram_list,is_complaint,is_suggestion
674,45de5edb-a0ed-4c51-9e5f-157dcb54bb07,2022-04-25 20:09:46.418593,940801451358,34569768,28817,classic,1,в приложении у меня стоит темный режим при это...,,,,True,"[в, приложение, у, я, стоить, темный, режим, п...",в приложение у я стоить темный режим при это р...,"[(в, приложение), (приложение, у), (у, я), (я,...","[(в, приложение, у), (приложение, у, я), (у, я...","[(в, приложение, у, я), (приложение, у, я, сто...",False,False
1233,ad6b26f4-5f22-41e3-bdbf-718c32b32977,2022-05-23 14:03:29.689962,900219450329,38890936,199913,transaction,2,jusan спасибо только колл центр по чаще бы под...,,,,False,"[jusan, спасибо, только, колл, центр, по, част...",jusan спасибо только колл центр по частый бы п...,"[(jusan, спасибо), (спасибо, только), (только,...","[(jusan, спасибо, только), (спасибо, только, к...","[(jusan, спасибо, только, колл), (спасибо, тол...",False,False
3040,fb154139-855e-4e62-9358-7e9f4d7f450c,2022-06-29 18:32:25.436171,870924350674,44709025,657596,classic,4,быстрые и безпроцентные переводы это огромный ...,,,,False,"[быстрый, и, безпроцентный, перевод, это, огро...",быстрый и безпроцентный перевод это огромный п...,"[(быстрый, и), (и, безпроцентный), (безпроцент...","[(быстрый, и, безпроцентный), (и, безпроцентны...","[(быстрый, и, безпроцентный, перевод), (и, без...",False,False
3175,3d6e489c-6095-498c-bffe-75eb24a47f4d,2022-06-29 19:23:07.856432,900105001048,44769908,705193,classic,5,по чаще давали бы еще кредиты тогда было бы су...,,,,False,"[по, частый, давать, бы, еще, кредит, тогда, б...",по частый давать бы еще кредит тогда быть бы с...,"[(по, частый), (частый, давать), (давать, бы),...","[(по, частый, давать), (частый, давать, бы), (...","[(по, частый, давать, бы), (частый, давать, бы...",False,True
3296,eac1bc5b-191c-4711-b8a6-dc07e5e3dde3,2022-07-02 12:35:57.824955,850130401028,45309919,796368,classic,2,в принципе приложение норм расстраивает частые...,,,,True,"[в, принцип, приложение, норма, расстраивать, ...",в принцип приложение норма расстраивать частый...,"[(в, принцип), (принцип, приложение), (приложе...","[(в, принцип, приложение), (принцип, приложени...","[(в, принцип, приложение, норма), (принцип, пр...",True,False
3453,ffebf2da-98bf-424a-bc4a-5aadab82e40a,2022-06-29 18:57:35.394727,960128350339,44722372,667116,classic,4,частые сбои в приложении а так все весьма удобно,,,,False,"[частый, сбой, в, приложение, а, так, весь, ве...",частый сбой в приложение а так весь весьма удо...,"[(частый, сбой), (сбой, в), (в, приложение), (...","[(частый, сбой, в), (сбой, в, приложение), (в,...","[(частый, сбой, в, приложение), (сбой, в, прил...",False,False
3663,d89198a4-d6de-4770-a403-1df6e97617c1,2022-06-29 20:31:53.990764,040604550052,44831654,756097,classic,3,частые ошибки в приложении медленный перевод д...,,,,False,"[частый, ошибка, в, приложение, медленный, пер...",частый ошибка в приложение медленный перевод д...,"[(частый, ошибка), (ошибка, в), (в, приложение...","[(частый, ошибка, в), (ошибка, в, приложение),...","[(частый, ошибка, в, приложение), (ошибка, в, ...",False,False


In [465]:
df1.iloc[674].lemmatized_string
#'хотеться бы', 'быть бы', 'можно быть', 'хотеться', 'можно'

'в приложение у я стоить темный режим при это раздел магазин и инвестиция белый цвет вы что издеваться что ли как у вы половина приложение черный цвет а половина белый из за что глаз реально больно я даже молчать про ваш частый техработ но блин приложение двухцветной атас '

In [ ]:
df_ru['text'].loc[0].split()

In [ ]:
dfwords.loc[0]

# 

# 

# Model

In [622]:
conn = gp_connect()
query = f"""SELECT * FROM public.questions where channel_id = 'omnichat-adapter-vn-BRB_RU' and timestamp > '2020-01-05' and question like '%филиал%' limit 3000"""
df_mo = psql.read_sql(query, conn)
conn.close()
df_mo

/miniconda3/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,log_id,question,from_state,to_state,score,pattern,effective_pattern,location,fast_reply,session_id,...,parse_tree,conflicts,class_id,confidence,fusion,weight_rb,weight_ml,score_norm,to_state_title_abs_path,level_result
0,5249311,здравствуйте я звоню из город караганда вчера ...,/welcome,/link-dialog/1/1.1,0.11,* {($kazakhCities|$city|$jysanBank) * ($branch...,* караганд* * есть * с банкомат* *,,false,317bad53-eb72-46e9-9b7c-2460661ec9ae,...,"{'tag': 'root', 'text': 'здравствуйте я звоню ...","[{'score': 0.05333333333333338, 'fusion': 0.09...",None,0.0,0.20,1.0,0.0,0.20,/Ссылки/Запросы/Адреса и график работы отделений,0.11
1,5254136,здравствуйте я хотела узнать вот филиал по мы ...,/welcome,/привет,0.20,* $hello *,* здравствуйте *,,false,14b4b967-db0c-4984-9465-10e443d67f28,...,"{'tag': 'root', 'text': 'здравствуйте я хотела...","[{'score': 0.17050847457627122, 'fusion': 0.33...",None,0.0,0.40,1.0,0.0,0.40,/привет,0.20
2,5254176,здравствуйте я хотела узнать мы можем филиал т...,/привет,/привет,0.12,* $hello *,* здравствуйте *,,false,14b4b967-db0c-4984-9465-10e443d67f28,...,"{'tag': 'root', 'text': 'здравствуйте я хотела...","[{'score': 0.0970192307692308, 'fusion': 0.333...",None,0.0,0.40,1.0,0.0,0.40,/привет,0.12
3,5278209,девушка была заблокирована карта отр сейчас он...,/moreQuestions,/faq/8/8.1,0.23,* {{$transfer [$money]} * ($card|$score|$depos...,* заблокирова* карт* * атф* * перевест* *,,false,1e7a086d-f614-4623-89a9-f13071882919,...,"{'tag': 'root', 'text': 'девушка была заблокир...","[{'score': 0.12226086956521741, 'fusion': 0.11...",None,0.0,0.22,1.0,0.0,0.22,/faq/ATF/запросы про атф,0.23
4,5259073,у меня нужно соединить филиала да правильно оп...,/welcome,/4,0.19,* {(переклю*|перевед*|перевод*|перевест*|соед*...,* соед* * оператор* *,,false,a5498a22-271f-4032-922c-66a97e92c03b,...,"{'tag': 'root', 'text': 'у меня нужно соединит...","[{'score': 0.1547692307692308, 'fusion': 0.452...",None,0.0,0.55,1.0,0.0,0.55,/по просьбе кл.перевод на оператора,0.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,16935835,насчет филиалы в городе актау,/4,/link-dialog/1/1.1,0.44,* {($address|$located|$working|$dayNight|$cash...,* филиал* * город* *,,false,522e0be1-96c0-4f80-82b8-fd36a1823f04,...,"{'tag': 'root', 'text': 'насчет филиалы в горо...",[],None,0.0,1.00,1.0,0.0,1.00,/Ссылки/Запросы/Адреса и график работы отделений,0.44
2996,16990060,где я могу снять хотела снять доллары деньги в...,/welcome,/schet-balance/1/1.6/1.6.2,0.15,* {(снят*|обналич*|сним*) * ($money|$kzMoney) ...,* деньг* * снят* сегодн* *,,false,06da00cc-950a-4b82-ab3f-98e44aeec22e,...,"{'tag': 'root', 'text': 'где я могу снять хоте...","[{'score': 0.14202020202020205, 'fusion': 0.27...",None,0.0,0.30,1.0,0.0,0.30,"/Тематика ""Баланс по текущему и карт счетам""/б...",0.15
2997,16990286,у меня интересует снятие где я могу в каком фи...,/schet-balance/1/1.1,/schet-balance/1/1.1/1.1.1,0.00,*,*,,false,06da00cc-950a-4b82-ab3f-98e44aeec22e,...,"{'tag': 'root', 'text': 'у меня интересует сня...","[{'score': 0.1986885245901639, 'fusion': 0.573...",None,0.0,0.00,0.0,1.0,0.00,"/Тематика ""Баланс по текущему и карт счетам""/б...",0.00
2998,17057452,у меня онлайн отделения в макинске филиал в jmart,/noMatch,/link-dialog/1/1.1,0.34,* {($address|$located|$working|$dayNight|$cash...,* отделен* * макинск* *,,false,6365535d-979f-4937-8938-6176cb5b6e71,...,"{'tag': 'root', 'text': 'у меня онлайн отделен...",[],None,0.0,1.00,1.0,0.0,1.00,/Ссылки/Запросы/Адреса и график работы отделений,0.34


In [608]:
df_mo

,log_id,question,from_state,to_state,score,pattern,effective_pattern,location,fast_reply,session_id,...,parse_tree,conflicts,class_id,confidence,fusion,weight_rb,weight_ml,score_norm,to_state_title_abs_path,level_result
0,19129729,мы хотели оформить товарный кредит в рассрочку...,/moreQuestions,/express-credit/4,0.42,* {(дайте|взять|возьму|$order|получ*|выда*|*да...,* оформи* товар* * рассрочк* *,,false,cae6db08-d47c-4acd-b79c-dfa8c8a2603a,...,"{'tag': 'root', 'text': 'мы хотели оформить то...","[{'score': 0.3728260869565217, 'fusion': 0.405...",None,0.0,0.45,1.0,0.0,0.45,"/Тематика ""Экспресс кредит в МП""/перевод на оп...",0.42
1,19298924,товарный кредит меня интересует,/express-credit/1/1.2,/express-credit/1/1.2/1.2.2,0.40,* (товар*|товар* [кредит|кридит]) *,* товар* кредит *,,false,cfaad49a-d321-40dc-bd04-51c8cea6a70c,...,"{'tag': 'root', 'text': 'товарный кредит меня ...","[{'score': 0.3960714285714286, 'fusion': 0.648...",None,0.0,1.00,1.0,0.0,1.00,"/Тематика ""Экспресс кредит в МП""/Виды кредитов...",0.40
2,19150576,товарный кредит,/express-credit/1/1.2,/express-credit/1/1.2/1.2.2,0.79,* (товар*|товар* [кредит|кридит]) *,* товар* кредит *,,false,12bd6fad-b523-4051-97db-5fb86a84e58e,...,"{'tag': 'root', 'text': 'товарный кредит'}","[{'score': 0.7921428571428571, 'fusion': 0.648...",None,0.0,1.00,1.0,0.0,1.00,"/Тематика ""Экспресс кредит в МП""/Виды кредитов...",0.79
3,19176677,товарный кредит,/express-credit/1/1.2,/express-credit/1/1.2/1.2.2,0.79,* (товар*|товар* [кредит|кридит]) *,* товар* кредит *,,false,3e44ff90-1084-437d-8648-7b4ed0272bad,...,"{'tag': 'root', 'text': 'товарный кредит'}","[{'score': 0.7921428571428571, 'fusion': 0.648...",None,0.0,1.00,1.0,0.0,1.00,"/Тематика ""Экспресс кредит в МП""/Виды кредитов...",0.79
4,19197500,по оформлению товарный кредит 2 раза оформился,/4,/express-credit/1,0.43,* {($jmart|магазин*|взять|возьму|$order|получ*...,* оформл* товар* кредит* *,,false,f02c027a-c10d-4e2e-9ca0-4403ac95d90a,...,"{'tag': 'root', 'text': 'по оформлению товарны...",[],None,0.0,0.74,1.0,0.0,0.74,"/Тематика ""Экспресс кредит в МП""/Виды кредитов",0.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1768,19085528,у меня есть товарный кредит мне нужно узнать г...,/4,/credit-balance/1/1.2,0.24,* {$mine * кредит* * (закрылся|есть|остал*) [$...,* у меня есть * кредит* *,,false,705eed5a-8281-451d-943f-a8db311ae385,...,"{'tag': 'root', 'text': 'у меня есть товарный ...","[{'score': 0.1760317460317461, 'fusion': 0.345...",None,0.0,0.47,1.0,0.0,0.47,/Баланс по кредитам/Инфо по действующему креди...,0.24
1769,19098006,товарный кредит,/first_check/check_transfer/question,/express-credit/1,0.79,* $marketable *,* товар* кредит* *,,false,e5ff2497-6226-4385-93a8-7794be969304,...,"{'tag': 'root', 'text': 'товарный кредит', 'in...",[],None,0.0,0.65,1.0,0.0,0.65,"/Тематика ""Экспресс кредит в МП""/Виды кредитов",0.79
1770,19098021,товарный кредит,/express-credit/1,/express-credit/1,0.79,* $marketable *,* товар* кредит* *,,false,e5ff2497-6226-4385-93a8-7794be969304,...,"{'tag': 'root', 'text': 'товарный кредит', 'in...",[],None,0.0,0.65,1.0,0.0,0.65,"/Тематика ""Экспресс кредит в МП""/Виды кредитов",0.79
1771,19098043,товарный кредит связаться с консультантом,/express-credit/1,/express-credit/1,0.30,* $marketable *,* товар* кредит* *,,false,e5ff2497-6226-4385-93a8-7794be969304,...,"{'tag': 'root', 'text': 'товарный кредит связа...","[{'score': 0.29891891891891886, 'fusion': 0.39...",None,0.0,0.39,1.0,0.0,0.39,"/Тематика ""Экспресс кредит в МП""/Виды кредитов",0.30


In [575]:
df_model = pd.DataFrame(columns=['question', 'topic'])

In [604]:
df_model

,question,topic
0,перевод денег из казахстана в кредит какая ком...,transfer
1,я не переводятся со счета на счет деньги,transfer
2,нет не 300 переводы баланс карты,transfer
3,у вас у вас тарифы и переводы,transfer
4,мне не переводятся деньги со счета на счет,transfer
...,...,...
11995,у меня получается сегодня платеж по кредиту 24...,payments
11996,как оплатить кредит,payments
11997,касательно блокировке оплата не проходит оплат...,payments
11998,оплата кредита,payments


In [592]:
# mobile_app = {'прилож'}
# transfers = {'перевод'} +
# debit_card = {'дебет'} +
# loan = {'кредит'} 
# product_loan = {'товарный кредит'} 1800
# deposit = {'депозит'} +
# payments = {'оплат'} +
# invest = {'инвест'} 400
# garant = {'гарант'} 900
# jusan_shop = {'магаз'}
# jmobile = {'mobile'} 1400
# bonuses = {'бонус'}
# qr = {'киюар', 'qr', 'кюар'} 1066
# call_center = {'кол'}
# branch = {'филиал или отделение'}
# employee = {'работника оператор'}
# junior = {'junior'} 720
df_mo['topic'] = 'payments'
df_mo = df_mo[['question', 'topic']]
df_model = df_model.append(df_mo)
df_model = df_model.reset_index()
del df_model['index']

/tmp/ipykernel_4026/697252415.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_model = df_model.append(df_mo)
